In [151]:
import ipynb_helpers as helpers
import pandas as pd

In [2]:
api_key = helpers.get_env_var("AEMET_API_KEY")
api_key[:3]

'eyJ'

In [3]:
url = 'https://opendata.aemet.es/AEMET_OpenData_specification.json'

In [4]:
response = helpers.make_request(url=url, method='get', response_format='json')

In [148]:
path_data = response[0]['paths']

In [122]:
param_data = {}
for key, val in path_data.items():
    if 'parameters' in path_data[key].get('get', {}):
        params = [p for p in path_data[key]['get']['parameters']]
        for param in params:
            name = param['name']
            if name not in param_data:
                param_data[name] = [param['description']]

In [135]:
param_df = pd.DataFrame(param_data).T.reset_index()
param_df.columns = ['name', 'description']
param_df.to_csv('datasets/params_raw.txt', index=False)

In [149]:
response_data = path_data['/api/avisos_cap/ultimoelaborado/area/{area}']['get']['responses']
response_codes = response_data.keys()
response_descriptions = [response_data[code]['description'] for code in response_codes]
response_df = pd.DataFrame({
    'response_code': response_codes,
    'description': response_descriptions,
})
response_df.to_csv('datasets/response_codes.csv', index=False)

In [150]:
endpoints = list(path_data.keys())
endpoint_data = [path_data[endpoint]['get'] for endpoint in endpoints]
endpoint_df = pd.DataFrame(endpoint_data, index=endpoints).reset_index()
endpoint_df = endpoint_df.drop(columns=['responses', 'security', 'description', 'operationId'])
endpoint_df['tags'] = endpoint_df['tags'].apply(lambda x: x[0])
endpoint_df['parameters'] = endpoint_df['parameters'].apply(lambda x: [param['name'] for param in x] if isinstance(x, list) else [])
endpoint_df.rename(columns={'index': 'endpoint', 'tags': 'tag'}, inplace=True)
endpoint_df.head()

,endpoint,tag,summary,parameters
0,/api/avisos_cap/ultimoelaborado/area/{area},avisos_cap,Avisos de Fenómenos Meteorológicos Adversos. Ú...,[area]
1,/api/avisos_cap/archivo/fechaini/{fechaIniStr}...,avisos_cap,Avisos de Fenómenos Meteorológicos Adversos. A...,"[fechaIniStr, fechaFinStr]"
2,/api/incendios/mapasriesgo/estimado/area/{area},indices-incendios,Mapa de niveles de riesgo estimado meteorológi...,[area]
3,/api/incendios/mapasriesgo/previsto/dia/{dia}/...,indices-incendios,Mapa de niveles de riesgo previsto meteorológi...,"[dia, area]"
4,/api/maestro/municipio/{municipio},maestro,Información específica municipio.,[municipio]


In [147]:
endpoint_df.to_csv('datasets/endpoints.csv', index=False)

In [153]:
playas_url = 'https://www.aemet.es/documentos/es/eltiempo/prediccion/playas/Playas_codigos.csv'
playas_data = helpers.make_request(url=playas_url, method='get', response_format='text')
with open('datasets/playas_data.csv', 'w') as file:
    file.write(playas_data[0])